# ShowdownIO
ShowdownIO is a Python-Selenium-based interface for autonomous battle monitoring and play on Pokémon Showdown. With an instance of the ShowdownIO class, you can automatically participate in or just collect information about battles happening on the world's most popular Pokémon battle simulator. 



## Setup

### Dependencies
ShowdownIO depends on the [Selenium Python bindings](https://selenium-python.readthedocs.io/) and a [driver](https://selenium-python.readthedocs.io/installation.html#drivers) to interface with the chosen browser (we currently prefer Firefox) in the home working path.

In [56]:
# selenium
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.support import ui
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
import os

### Constants

In [57]:
# for driver setup
site_url = 'https://play.pokemonshowdown.com/'
driver_path = os.path.join(os.getcwd(), 'geckodriver.exe')

# for interacting with site html
login = "//button[@name='login']"
loggedin = "//span[@class='username']"
openoptions = "//button[@name='openOptions']"
register = "//button[@name='register']"
enterusername = "//input[@name='username']"
enterpassword = "//input[@name='password']"
confirmpassword = "//input[@name='cpassword']"
captcha = "//input[@name='captcha']"
submit = "//button[@type='submit']"
error = "//p[@class='error']"
close = "//button[@name='close']"
hometab = '//div[@class="tabbar maintabbar"]/div/ul'
startbattle = '//button[@class="button mainmenu1 big"]'
registered = "The name you chose is registered."
wrongpassword = "Wrong password."
alreadyusing = "Someone is already using the name "

## Helper Functions

In [58]:
def confirmClick(element):
    "Keeps trying to click element and returns True upon success"
    
    while True:
        try:
            element.click()
            return True
        except ElementNotInteractableException:
            pass

## The ShowdownIO Class
Users will use instances of this class to create and operate their bots.

In [59]:
class ShowdownIO(object):
    """An interface for autonomous battle monitoring and play on Pokémon Showdown using the Selenium Python bindings.
    
    Attributes:
        driver_path: path to the web driver associated with class
        site_url: 'https://play.pokemonshowdown.com/'
        battles: list of representations of all live battles associated with account
    """

    def __init__(self, username, password):
        "Inits ShowdownIO, logging into PS w/ specified username and password."
        
        # start browser and open site
        self.driver = webdriver.Firefox(FirefoxProfile(), executable_path=driver_path)
        self.wait = ui.WebDriverWait(self.driver, 10)
        self.driver.get(site_url)
        self.wait.until(lambda _: self.xpath(login) or self.xpath(loggedin))
        
        # log in if not logged in
        if self.xpath(login):
            self.xpath(login).click()
            self.xpath(enterusername).send_keys(username)
            self.xpath(enterusername).send_keys(Keys.RETURN)
            self.wait.until(lambda _: username in self.xpath().text or self.xpath(error))
            
            # possibilities: username used, registered, unregistered
            # username is registered: submit password and raise error if failure
            if self.xpath(enterpassword):
                self.xpath(enterpassword).send_keys(password)
                self.xpath(enterpassword).send_keys(Keys.RETURN)
                self.wait.until(lambda _: self.xpath(error) or not self.xpath(login))
                if self.xpath(error):
                    raise ValueError(wrongpassword)
                    
            # someone using username now: raise error
            elif self.xpath(error):
                raise ValueError(alreadyusing)
           
            # username is unregistered: register with specified password
            else:
                self.xpath(openoptions).click()
                self.wait.until(lambda _: self.xpath(register))
                self.xpath(register).click()
                self.wait.until(lambda _: self.xpath(enterpassword))
                self.xpath(enterpassword).send_keys(password)
                self.xpath(confirmpassword).send_keys(password)
                self.xpath(captcha).send_keys("pikachu")
                self.xpath(submit).click()
                self.wait.until(lambda _: not self.xpath(login))
                self.xpath(close).click()
           
        # initialize representation of all "live" battles
        self.battles = []
        
    def start_battle(self):
        "Starts a new battle; only resolves if battle successfully started."
        
        # first check # of tabs
        original_tab_count = len(self.driver.find_elements_by_xpath(hometab))
        
        # switch to home tab and start battle
        self.xpath(hometab).click()
        self.wait.until(lambda _: confirmClick(self.xpath(startbattle)))
        
        # wait until # of tabs increases by 1
        self.wait.until(lambda _: original_tab_count < len(self.driver.find_elements_by_xpath(hometab)))
    
    def xpath(self, xpath="//body"):
        "Returns first element with specified xpath on driver's current webpage"
        try:
            return self.driver.find_element_by_xpath(xpath)
        except NoSuchElementException:
            return None

## Testing

In [60]:
bot = ShowdownIO(username='battlebot9', password='password')